In [1]:
# ====================================
# 1. Install dan Import Library
# ====================================
!pip install geopandas folium scikit-learn --quiet

import pandas as pd
import numpy as np
import folium
from folium.plugins import MarkerCluster
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


In [2]:
# ====================================
# 2. Load Dataset (pastikan sudah diunggah ke Colab)
# ====================================
curah_hujan = pd.read_csv("1_Bandung_curah_hujan_full.csv")
non_spasial = pd.read_csv("2_Bandung_non_spasial_full.csv")
riwayat_banjir = pd.read_csv("3_Bandung_riwayat_banjir_full.csv")
penggunaan_lahan = pd.read_csv("4_Bandung_penggunaan_lahan_full.csv")
prediksi_banjir = pd.read_csv("5_Bandung_prediksi_banjir_full.csv")


In [3]:
# ====================================
# 3. Preprocessing dan Gabungkan Data
# ====================================
curah_mean = curah_hujan.groupby("kecamatan")["curah_hujan_mm"].mean().reset_index()
banjir_total = riwayat_banjir.groupby("kecamatan")["jumlah_banjir"].sum().reset_index()

df = curah_mean.merge(non_spasial, on="kecamatan") \
               .merge(banjir_total, on="kecamatan") \
               .merge(penggunaan_lahan, on="kecamatan") \
               .merge(prediksi_banjir, on="kecamatan")

df['label'] = df['Prediksi_Banjir'].map({"Rendah": 0, "Sedang": 1, "Tinggi": 2})

In [4]:
# ====================================
# 4. Model Prediksi dengan Random Forest
# ====================================
X = df[["curah_hujan_mm", "jumlah_penduduk", "kepadatan_jiwa_km2",
        "jumlah_banjir", "permukiman_persen", "ruang_terbuka_hijau_persen"]]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Akurasi Model:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Akurasi Model: 0.0
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       4.0
           1       0.00      0.00      0.00       2.0
           2       0.00      0.00      0.00       1.0

    accuracy                           0.00       7.0
   macro avg       0.00      0.00      0.00       7.0
weighted avg       0.00      0.00      0.00       7.0



In [5]:

# ====================================
# 5. Koordinat Tiap Kecamatan Bandung
# ====================================
kec_coords = {
    "Andir": (-6.9131, 107.5733), "Antapani": (-6.9002, 107.6473), "Arcamanik": (-6.8989, 107.6790),
    "Astanaanyar": (-6.9266, 107.5997), "Babakan Ciparay": (-6.9355, 107.5777), "Bandung Kidul": (-6.9580, 107.6374),
    "Bandung Kulon": (-6.9275, 107.5620), "Bandung Wetan": (-6.9020, 107.6197), "Batununggal": (-6.9337, 107.6327),
    "Bojongloa Kaler": (-6.9432, 107.5784), "Bojongloa Kidul": (-6.9484, 107.6027), "Buahbatu": (-6.9475, 107.6467),
    "Cibeunying Kaler": (-6.8879, 107.6274), "Cibeunying Kidul": (-6.9024, 107.6442), "Cibiru": (-6.8865, 107.7163),
    "Cicendo": (-6.9001, 107.5958), "Cidadap": (-6.8585, 107.6001), "Cinambo": (-6.8991, 107.7007),
    "Coblong": (-6.8827, 107.6109), "Gedebage": (-6.9603, 107.7112), "Kiaracondong": (-6.9290, 107.6557),
    "Lengkong": (-6.9261, 107.6228), "Mandala Jati": (-6.8946, 107.6653), "Panyileukan": (-6.9047, 107.7126),
    "Rancasari": (-6.9542, 107.6950), "Regol": (-6.9436, 107.6145), "Sukajadi": (-6.8824, 107.5771),
    "Sukasari": (-6.8672, 107.5814), "Sumur Bandung": (-6.9171, 107.6091), "Ujungberung": (-6.8866, 107.7111)
}

df['lat'] = df['kecamatan'].map(lambda x: kec_coords.get(x, (0, 0))[0])
df['lon'] = df['kecamatan'].map(lambda x: kec_coords.get(x, (0, 0))[1])

In [17]:
# ====================================
# 6. Visualisasi Interaktif dengan Folium (cerah, petak, label, legenda)
# ====================================
import branca.colormap as cm

# Gunakan peta cerah dan profesional
m = folium.Map(location=[-6.9, 107.6], zoom_start=12, tiles='CartoDB Positron')

color_map = {0: 'green', 1: 'orange', 2: 'red'}
label_map = {0: 'Rendah', 1: 'Sedang', 2: 'Tinggi'}

# Plot kotak dan label di peta
for _, row in df.iterrows():
    bounds = [
        [row['lat'] - 0.004, row['lon'] - 0.004],
        [row['lat'] + 0.004, row['lon'] + 0.004]
    ]

    folium.Rectangle(
        bounds=bounds,
        color=color_map[row['label']],
        fill=True,
        fill_opacity=0.6,
        weight=1,
        popup=folium.Popup(
            f"""
            <b>Kecamatan:</b> {row['kecamatan']}<br>
            <b>Prediksi:</b> {label_map[row['label']]}<br>
            <b>Curah Hujan:</b> {row['curah_hujan_mm']} mm<br>
            <b>Jumlah Banjir:</b> {row['jumlah_banjir']}
            """, max_width=300)
    ).add_to(m)

    # Tambah label nama kecamatan
    folium.Marker(
        [row['lat'], row['lon']],
        icon=folium.DivIcon(html=f"""
            <div style='font-size:10pt; color:black; text-shadow: 1px 1px 2px white;'>
                <b>{row['kecamatan']}</b>
            </div>""")
    ).add_to(m)

# Tambah legenda
legend_html = '''
<div style="
    position: fixed;
    bottom: 40px;
    left: 40px;
    width: 170px;
    height: 120px;
    background-color: rgba(255, 255, 255, 0.9);
    border:2px solid #444;
    z-index:9999;
    font-size:14px;
    padding:10px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.5);
">
<b>Legenda Risiko Banjir</b><br>
<i style="background:green;width:12px;height:12px;display:inline-block;"></i>&nbsp;Rendah<br>
<i style="background:orange;width:12px;height:12px;display:inline-block;"></i>&nbsp;Sedang<br>
<i style="background:red;width:12px;height:12px;display:inline-block;"></i>&nbsp;Tinggi<br>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))





In [18]:
# ====================================
# 7. Simpan Peta Sebagai File HTML & Unduh
# ====================================
m.save("peta_prediksi_banjir_bandung.html")

# Untuk mengunduh ke lokal
from google.colab import files
files.download("peta_prediksi_banjir_bandung.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>